In [1]:
from dataset import *
from czii_helper import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import random
import torchvision.transforms as T
import cv2
import math
import cc3d
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import segmentation_models_pytorch as smp

In [2]:
train_dir = '../input/czii-cryo-et-object-identification/train/static/ExperimentRuns/'
mask_dir = '../input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/'
DEBAG = False
train_id = ["TS_6_4", "TS_6_6", "TS_69_2", "TS_73_6", "TS_86_3"]
if DEBAG:
    valid_id = ["TS_5_4",'TS_99_9']
valid_id = ["TS_5_4",'TS_99_9']

In [12]:
import albumentations as A
transform = A.Compose([
    A.HorizontalFlip(p=0.5),          # 50% の確率で水平反転
    A.RandomBrightnessContrast(p=0.2), # 明るさとコントラストの調整
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # 3チャンネル用正規化
    A.Resize(640, 640), 
])

class Dataset2D(Dataset):
    def __init__(self, IDs, train_dir, mask_dir, transform=None):
        self.IDs = IDs  # 使用するIDのリスト
        self.train_dir = train_dir
        self.mask_dir = mask_dir
        self.transform = transform

        # IDごとにデータとマスクを読み込む
        self.data = []
        self.masks = []
        for ID in self.IDs:
            data = read_one_data(ID, self.train_dir)
            mask = np.load(f"../input/mask/train_label_{ID}.npy")
            self.data.append(data)
            self.masks.append(mask)

        # データをまとめて1つのNumPy配列にする
        self.data = np.concatenate(self.data, axis=0)
        self.masks = np.concatenate(self.masks, axis=0)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        data = self.data[idx]   # (630, 630)
        mask = self.masks[idx]  # (630, 630)

        # ✅ データを3チャンネルに変換
        data = np.repeat(data[..., None], 3, axis=2)  # (630, 630, 3)

        # ✅ マスクは1チャンネルのまま
        mask = mask[..., None]  # (630, 630, 1)

        # ✅ Albumentationsでリサイズと変換
        if self.transform:
            augmented = self.transform(image=data, mask=mask)
            data = augmented['image']
            mask = augmented['mask']

        data = np.transpose(data, (2, 0, 1))  # (3, 640, 640)
        mask = np.transpose(mask, (2, 0, 1))  # (1, 640, 640)

        data = torch.tensor(data, dtype=torch.float32)
        mask = torch.tensor(mask, dtype=torch.float32)

        return data, mask


In [13]:
train_dataset = Dataset2D(train_id, train_dir, mask_dir, transform=transform)
valid_dataset = Dataset2D(valid_id, train_dir, mask_dir, transform=None)

# DataLoaderを作成
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False)

In [14]:
for batch_data, batch_mask in train_loader:
    print(batch_data.shape)  # (4, 630, 630)
    print(batch_mask.shape)  # (4, 630, 630)
    break

torch.Size([4, 3, 640, 640])
torch.Size([4, 1, 640, 640, 7])


In [15]:
class TwoConvBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, middle_channels, kernel_size = 3, padding="same")
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.rl = nn.ReLU()
        self.conv2 = nn.Conv2d(middle_channels, out_channels, kernel_size = 3, padding="same")
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.rl(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.rl(x)
        return x

class UpConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size = 2, padding="same")
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.up(x)
        x = self.bn1(x)
        x = self.conv(x)
        x = self.bn2(x)
        return x

class UNet_2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.TCB1 = TwoConvBlock(3, 64, 64)
        self.TCB2 = TwoConvBlock(64, 128, 128)
        self.TCB3 = TwoConvBlock(128, 256, 256)
        self.TCB4 = TwoConvBlock(256, 512, 512)
        self.TCB5 = TwoConvBlock(512, 1024, 1024)
        self.TCB6 = TwoConvBlock(1024, 512, 512)
        self.TCB7 = TwoConvBlock(512, 256, 256)
        self.TCB8 = TwoConvBlock(256, 128, 128)
        self.TCB9 = TwoConvBlock(128, 64, 64)
        self.maxpool = nn.MaxPool2d(2, stride = 2)
        
        self.UC1 = UpConv(1024, 512) 
        self.UC2 = UpConv(512, 256) 
        self.UC3 = UpConv(256, 128) 
        self.UC4= UpConv(128, 64)

        self.conv1 = nn.Conv2d(64, 7, kernel_size = 1)
        self.soft = nn.Softmax(dim = 1)

    def forward(self, x):
        x = self.TCB1(x)
        x1 = x
        x = self.maxpool(x)

        x = self.TCB2(x)
        x2 = x
        x = self.maxpool(x)

        x = self.TCB3(x)
        x3 = x
        x = self.maxpool(x)

        x = self.TCB4(x)
        x4 = x
        x = self.maxpool(x)

        x = self.TCB5(x)

        x = self.UC1(x)
        x = torch.cat([x4, x], dim = 1)
        x = self.TCB6(x)

        x = self.UC2(x)
        x = torch.cat([x3, x], dim = 1)
        x = self.TCB7(x)

        x = self.UC3(x)
        x = torch.cat([x2, x], dim = 1)
        x = self.TCB8(x)

        x = self.UC4(x)
        x = torch.cat([x1, x], dim = 1)
        x = self.TCB9(x)

        x = self.conv1(x)

        return x

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
unet = UNet_2D().to(device)
optimizer = optim.Adam(unet.parameters(), lr=0.001)

In [17]:
TverskyLoss = smp.losses.TverskyLoss(mode='multilabel', log_loss=False)
BCELoss     = smp.losses.SoftBCEWithLogitsLoss()
def criterion(pred,target):
    return 0.5*BCELoss(pred, target) + 0.5*TverskyLoss(pred, target)

In [18]:
history = {"train_loss": []}
n = 0
m = 0

for epoch in range(15):
  train_loss = 0
  val_loss = 0

  unet.train()
  for i, data in enumerate(train_loader):
    inputs, labels = data[0].to(device), data[1].to(device)
  
    optimizer.zero_grad()
    outputs = unet(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    history["train_loss"].append(loss.item())
    n += 1
    print(f"epoch:{epoch+1}  index:{i+1}  train_loss:{train_loss/n:.5f}")
    n = 0
    train_loss = 0
    train_acc = 0


  unet.eval()
  with torch.no_grad():
    for i, data in enumerate(val_loader):
      inputs, labels = data[0].to(device), data[1].to(device)
      outputs = unet(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item()
      m += 1
      print(f"epoch:{epoch+1}  index:{i+1}  val_loss:{val_loss/m:.5f}")
      m = 0
      val_loss = 0
      val_acc = 0

  torch.save(unet.state_dict(), f"./train_{epoch+1}.pth")
print("finish training")

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 15.64 GiB of which 17.50 MiB is free. Including non-PyTorch memory, this process has 15.60 GiB memory in use. Of the allocated memory 15.31 GiB is allocated by PyTorch, and 107.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)